In [7]:
import os
from PIL import Image
import torch
from torchvision import transforms
from tqdm import tqdm

In [8]:
# Paths
root_folder = "../Resized_argued_MyDataset"  # Root folder containing the dataset
output_folder = "../Resized_argued_normed_MyDataset"  # Folder to save normalized images

categories = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

In [9]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalization transform
normalization_transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to tensor and scales to [0, 1]
])

# Convert tensor back to image for saving
to_image_transform = transforms.ToPILImage()

Using device: cuda


In [10]:
# Ensure output folder structure
def ensure_output_structure(root_folder, output_folder, categories):
    os.makedirs(output_folder, exist_ok=True)
    for category in categories:
        category_path = os.path.join(output_folder, category)
        os.makedirs(category_path, exist_ok=True)

ensure_output_structure(root_folder, output_folder, categories)

In [11]:
# GPU-accelerated normalization
def normalize_images_gpu(root_folder, output_folder, categories, device):
    for category in categories:
        input_folder = os.path.join(root_folder, category)
        output_category_folder = os.path.join(output_folder, category)
        
        images = [img for img in os.listdir(input_folder) if img.lower().endswith(('png', 'jpg', 'jpeg'))]
        
        for img_name in tqdm(images, desc=f"Normalizing {category}"):
            input_img_path = os.path.join(input_folder, img_name)
            output_img_path = os.path.join(output_category_folder, img_name)
            
            # Load image using PIL
            img = Image.open(input_img_path).convert("RGB")
            
            # Apply normalization (move to GPU)
            img_tensor = normalization_transform(img).unsqueeze(0).to(device)  # Add batch dimension
            img_tensor = img_tensor.squeeze(0)  # Remove batch dimension
            
            # Move tensor back to CPU and save as image
            normalized_img = to_image_transform(img_tensor.cpu())
            normalized_img.save(output_img_path)

In [12]:
normalize_images_gpu(root_folder, output_folder, categories, device)

Normalizing Surprise: 100%|██████████| 13239/13239 [04:08<00:00, 53.32it/s]
